incomplete

In [1]:
from gssutils import *

if is_interactive():
    import requests
    from cachecontrol import CacheControl
    from cachecontrol.caches.file_cache import FileCache
    from cachecontrol.heuristics import LastModified
    from pathlib import Path

    session = CacheControl(requests.Session(),
                           cache=FileCache('.cache'),
                           heuristic=LastModified())

    sourceFolder = Path('in')
    sourceFolder.mkdir(exist_ok=True)

    inputURL = 'https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/752515/AdultSubstanceMisuseNDTMSDataTables2017-18.xlsx'
    inputFile = sourceFolder / 'AdultSubstanceMisuseNDTMSDataTables2017-18.xlsx'
    response = session.get(inputURL)
    with open(inputFile, 'wb') as f:
      f.write(response.content)    

In [2]:
tab = loadxlstabs(inputFile, sheetids='Table 5.2.1')[0]

Loading in\AdultSubstanceMisuseNDTMSDataTables2017-18.xlsx which has size 272149 bytes
Table names: ['Table 5.2.1']


https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/752515/AdultSubstanceMisuseNDTMSDataTables2017-18.xlsx

In [3]:
observations = tab.excel_ref('B4').expand(DOWN).expand(RIGHT).is_not_blank()
observations

{<C5 4.0>, <C8 7.0>, <D4 8.0>, <D6 7.0>, <F4 1991.0>, <B8 4.0>, <E6 12.0>, <F5 4716.0>, <E5 12.0>, <B5 4690.0>, <E4 19.0>, <D7 44.0>, <D5 10.0>, <D8 0.0>, <C4 2.0>, <B4 1962.0>, <C7 25.0>, <B6 460.0>, <E8 0.0>, <B7 12427.0>, <E7 58.0>, <F7 12554.0>, <F6 480.0>, <F8 11.0>, <C6 1.0>}

In [5]:
intervention = tab.excel_ref('A4').expand(DOWN).is_not_blank()
intervention

{<A10 '*Individuals may have more than one type of intervention and so may appear on more than one row'>, <A5 'Structured day programme'>, <A7 'Structured intervention'>, <A6 'Residential rehabilitation'>, <A8 'Old YP intervention'>, <A4 'Inpatient detoxification'>}

In [6]:
clients = tab.excel_ref('B3').expand(RIGHT).is_not_blank()
clients

{<D3 'Non-opiate and Alcohol'>, <E3 'Alcohol only'>, <B3 'Opiate'>, <F3 'Total'>, <C3 'Non-opiate only'>}

In [7]:
Dimensions = [
            HDim(intervention,'Intervention',DIRECTLY,LEFT),
            HDim(clients,'Clients in treatment',CLOSEST,LEFT),
            HDimConst('Measure Type','Count'),
            HDimConst('Unit','People')            
            ]

In [8]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
# if is_interactive():
#     savepreviewhtml(c1)

In [9]:
new_table = c1.topandas()
new_table

,OBS,Intervention,Clients in treatment,Measure Type,Unit
0,1962.0,Inpatient detoxification,Opiate,Count,People
1,2.0,Inpatient detoxification,Non-opiate only,Count,People
2,8.0,Inpatient detoxification,Non-opiate and Alcohol,Count,People
3,19.0,Inpatient detoxification,Alcohol only,Count,People
4,1991.0,Inpatient detoxification,Total,Count,People
5,4690.0,Structured day programme,Opiate,Count,People
6,4.0,Structured day programme,Non-opiate only,Count,People
7,10.0,Structured day programme,Non-opiate and Alcohol,Count,People
8,12.0,Structured day programme,Alcohol only,Count,People
9,4716.0,Structured day programme,Total,Count,People


In [10]:
new_table = new_table[new_table['OBS'] != 0 ]

In [11]:
new_table.columns = ['Value' if x=='OBS' else x for x in new_table.columns]

In [12]:
new_table.head()

,Value,Intervention,Clients in treatment,Measure Type,Unit
0,1962.0,Inpatient detoxification,Opiate,Count,People
1,2.0,Inpatient detoxification,Non-opiate only,Count,People
2,8.0,Inpatient detoxification,Non-opiate and Alcohol,Count,People
3,19.0,Inpatient detoxification,Alcohol only,Count,People
4,1991.0,Inpatient detoxification,Total,Count,People


In [13]:
new_table.tail()

,Value,Intervention,Clients in treatment,Measure Type,Unit
18,58.0,Structured intervention,Alcohol only,Count,People
19,12554.0,Structured intervention,Total,Count,People
20,4.0,Old YP intervention,Opiate,Count,People
21,7.0,Old YP intervention,Non-opiate only,Count,People
24,11.0,Old YP intervention,Total,Count,People


In [14]:
new_table.dtypes

Value                   float64
Intervention             object
Clients in treatment     object
Measure Type             object
Unit                     object
dtype: object

In [15]:
new_table['Value'] = new_table['Value'].astype(str)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [16]:
new_table.head(3)

,Value,Intervention,Clients in treatment,Measure Type,Unit
0,1962.0,Inpatient detoxification,Opiate,Count,People
1,2.0,Inpatient detoxification,Non-opiate only,Count,People
2,8.0,Inpatient detoxification,Non-opiate and Alcohol,Count,People


In [18]:
new_table['Clients in treatment'] = new_table['Clients in treatment'].map(
    lambda x: {
        'Total' : 'All' 
        }.get(x, x))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [19]:
new_table.head(3)

,Value,Intervention,Clients in treatment,Measure Type,Unit
0,1962.0,Inpatient detoxification,Opiate,Count,People
1,2.0,Inpatient detoxification,Non-opiate only,Count,People
2,8.0,Inpatient detoxification,Non-opiate and Alcohol,Count,People


In [20]:
new_table = new_table[['Intervention','Clients in treatment','Measure Type','Value','Unit']]

In [21]:
if is_interactive():
    SubstancetinationFolder = Path('out')
    SubstancetinationFolder.mkdir(exist_ok=True, parents=True)
    new_table.to_csv(SubstancetinationFolder / ('table5.2.1.csv'), index = False)